# INFOLEG search tool

In [1]:
import pandas as pd
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# custom libs
import utils

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


## Download data

In [2]:
# download or load data (if already in the folder "data")
url = 'http://datos.jus.gob.ar/dataset/d9a963ea-8b1d-4ca3-9dd9-07a4773e8c23/resource/bf0ec116-ad4e-4572-a476-e57167a84403/download/base-infoleg-normativa-nacional.zip'
local_zip_path = 'data/base-infoleg-normativa-nacional.zip'

utils.create_dir_if_not_exists('data')

df = utils.download_and_read_zip(url, local_zip_path)

Directory already exists: data
File already exists: data/base-infoleg-normativa-nacional.zip
Extracted all files from data/base-infoleg-normativa-nacional.zip to data


In [3]:
df.head()

,id_norma,tipo_norma,numero_norma,clase_norma,organismo_origen,fecha_sancion,numero_boletin,fecha_boletin,pagina_boletin,titulo_resumido,titulo_sumario,texto_resumido,observaciones,texto_original,texto_actualizado,modificada_por,modifica_a
0,183290,Ley,S/N,NaN,HONORABLE CONGRESO DE LA NACION ARGENTINA,1853-11-09,NaN,NaN,NaN,ORGANIZACION DE LA HACIENDA Y CREDITO PUBLICO,ADMINISTRACION PUBLICA NACIONAL,ESTATUTO PARA LA ORGANIZACION DE LA HACIENDA Y...,LA PRESENTE LEY CORRESPONDE A LAS SANCIONADAS ...,http://servicios.infoleg.gob.ar/infolegInterne...,NaN,1,NaN
1,233172,Decreto,S/N,NaN,PODER EJECUTIVO NACIONAL (P.E.N.),1854-05-29,NaN,NaN,NaN,UNIVERSIDAD MAYOR DE SAN CARLOS - ESTABLECESE ...,EDUCACION SUPERIOR,LA UNIVERSIDAD MAYOR DE SAN CARLOS EN LA CIUDA...,PUBLICADA EN REGISTRO OFICIAL DE LA REPUBLICA ...,http://servicios.infoleg.gob.ar/infolegInterne...,NaN,0,NaN
2,209422,Decreto,S/N,NaN,PODER EJECUTIVO NACIONAL (P.E.N.),1854-09-26,NaN,NaN,NaN,SE MANDA PREPARAR EL PRESUPUESTO GENERAL PARA ...,PRESUPUESTO GENERAL DE LA ADMINISTRACION NACIONAL,ART. 1° LOS MINISTROS DEL GOBIERNO DE LA CONFE...,LA NUMERACION DE LA PRESENTE LEY CORRESPONDE A...,http://servicios.infoleg.gob.ar/infolegInterne...,NaN,3,NaN
3,280916,Ley,1,NaN,HONORABLE CONGRESO DE LA NACION ARGENTINA,1854-11-05,NaN,NaN,NaN,CREDITO FISCAL,FISCO NACIONAL,OBJETO CUMPLIDO-DESMONETIZACION DE LOS BILLETE...,PUBLICADA EN REGISTRO NACIONAL DE 1856,NaN,NaN,0,NaN
4,280915,Ley,2,NaN,HONORABLE CONGRESO DE LA NACION ARGENTINA,1854-11-07,NaN,NaN,NaN,NaN,NaN,AMBITO TEMPORAL CUMPLIDO-DERECHO ADICIONAL A L...,PUBLICADA EN REGISTRO NACIONAL DE 1856,NaN,NaN,0,NaN


## Process data

In [4]:
# Basic list of Spanish stopwords
basic_stop_words = ['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 
                    'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 
                    'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 
                    'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 
                    'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 
                    'esa', 'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 
                    'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosotros', 'vosotras', 
                    'os', 'mío', 'mía', 'míos', 'mías', 'tuyo', 'tuya', 'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas', 
                    'nuestro', 'nuestra', 'nuestros', 'nuestras', 'vuestro', 'vuestra', 'vuestros', 'vuestras', 'esos', 'esas']

# Function to preprocess text
def preprocess_text(text):
    # Convert text to lowercase
    text = str(text).lower()
    # Remove punctuation
    text = text.translate(text.maketrans('', '', string.punctuation))
    # Tokenize text by splitting on space
    word_tokens = text.split()
    # Remove stopwords
    filtered_tokens = [word for word in word_tokens if word not in basic_stop_words]
    # Join words back to text
    text = " ".join(filtered_tokens)
    return text

# Combine the text fields
df['combined_text'] = df['titulo_resumido'] + " " + df['titulo_sumario'] + " " + df['texto_resumido']

# Preprocess the combined text
df['preprocessed_text'] = df['combined_text'].apply(preprocess_text)

# Create a TfidfVectorizer object
vectorizer = TfidfVectorizer()

# Fit and transform the preprocessed text
tfidf_matrix = vectorizer.fit_transform(df['preprocessed_text'])

# Function to find the most similar documents given a query
def search_legal_data(query, vectorizer, tfidf_matrix, top_n=10):
    # Preprocess the query
    preprocessed_query = preprocess_text(query)
    # Vectorize the query
    query_vector = vectorizer.transform([preprocessed_query])
    # Compute cosine similarities
    cosine_similarities = linear_kernel(query_vector, tfidf_matrix).flatten()
    # Get the top_n most similar documents
    related_docs_indices = cosine_similarities.argsort()[:-top_n:-1]
    # Return the most similar documents with their similarity scores
    return df.iloc[related_docs_indices], cosine_similarities[related_docs_indices]

In [5]:
# Test the search function
query = "derechos de propiedad"
search_results, search_scores = search_legal_data(query, vectorizer, tfidf_matrix)
search_results['similarity_score'] = search_scores
search_results[['titulo_resumido', 'organismo_origen', 'texto_resumido', 'similarity_score']]

/var/folders/sl/2gx4wvjx5rg2w3mvybwh2dt00000gn/T/ipykernel_27812/86769655.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  search_results['similarity_score'] = search_scores


,titulo_resumido,organismo_origen,texto_resumido,similarity_score
150475,AGENTE DE LA PROPIEDAD INDUSTRIAL - APRUEBASE ...,INSTITUTO NACIONAL DE LA PROPIEDAD INDUSTRIAL,APRUEBASE EL REGLAMENTO PARA EL EJERCICIO DE L...,0.411357
295053,DELEGACION DE FACULTADES,PODER EJECUTIVO NACIONAL (P.E.N.),DELEGASE EN EL SEÑOR MINISTRO DE JUSTICIA Y DE...,0.373515
379008,INSCRIPCION,INSTITUTO NACIONAL DE LA PROPIEDAD INDUSTRIAL,INSCRIBASE AL EL SEÑOR JOSE LUIS SANCHEZ (DNI ...,0.360556
365541,INSCRIPCION,INSTITUTO NACIONAL DE LA PROPIEDAD INDUSTRIAL,INSCRIBASE A LA SEÑORA MARIA ALEJANDRA RODRIGU...,0.358589
379297,INSCRIPCION,INSTITUTO NACIONAL DE LA PROPIEDAD INDUSTRIAL,INSCRIBASE AL SEÑOR JUAN EDUARDO VERA (DNI N° ...,0.357703
363183,INSCRIPCION,INSTITUTO NACIONAL DE LA PROPIEDAD INDUSTRIAL,INSCRIBASE EL SEÑOR DIEGO NICOLAS GONZALEZ (DN...,0.356858
365142,INSCRIPCION,INSTITUTO NACIONAL DE LA PROPIEDAD INDUSTRIAL,INSCRIBASE AL SEÑOR WALTER HORACIO MARTINEZ (D...,0.354887
363268,INSCRIPCION,INSTITUTO NACIONAL DE LA PROPIEDAD INDUSTRIAL,INSCRIBASE AL SEÑOR ALEJANDRO JAVIER VEGA (DNI...,0.354865
379119,INSCRIPCION,INSTITUTO NACIONAL DE LA PROPIEDAD INDUSTRIAL,INSCRIBASE AL SEÑOR RAFAEL CORNEJO (DNI N° 33....,0.354517


In [6]:
search_results

,id_norma,tipo_norma,numero_norma,clase_norma,organismo_origen,fecha_sancion,numero_boletin,fecha_boletin,pagina_boletin,titulo_resumido,titulo_sumario,texto_resumido,observaciones,texto_original,texto_actualizado,modificada_por,modifica_a,combined_text,preprocessed_text,similarity_score
150475,115786,Resolución,101,NaN,INSTITUTO NACIONAL DE LA PROPIEDAD INDUSTRIAL,2006-04-18,30891.0,2006-04-24,20.0,AGENTE DE LA PROPIEDAD INDUSTRIAL - APRUEBASE ...,PROPIEDAD INDUSTRIAL,APRUEBASE EL REGLAMENTO PARA EL EJERCICIO DE L...,ABROGADA POR EL ARTICULO 1° DE LA RESOLUCION 1...,http://servicios.infoleg.gob.ar/infolegInterne...,http://servicios.infoleg.gob.ar/infolegInterne...,12,4.0,AGENTE DE LA PROPIEDAD INDUSTRIAL - APRUEBASE ...,agente propiedad industrial apruebase reglamen...,0.411357
295053,273998,Decreto,282,NaN,PODER EJECUTIVO NACIONAL (P.E.N.),2017-04-21,33610.0,2017-04-24,5.0,DELEGACION DE FACULTADES,MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS,DELEGASE EN EL SEÑOR MINISTRO DE JUSTICIA Y DE...,NaN,http://servicios.infoleg.gob.ar/infolegInterne...,NaN,1,2.0,DELEGACION DE FACULTADES MINISTERIO DE JUSTICI...,delegacion facultades ministerio justicia dere...,0.373515
379008,380234,Resolución,71,NaN,INSTITUTO NACIONAL DE LA PROPIEDAD INDUSTRIAL,2023-02-28,35122.0,2023-03-03,8.0,INSCRIPCION,INSTITUTO NACIONAL DE LA PROPIEDAD INDUSTRIAL,INSCRIBASE AL EL SEÑOR JOSE LUIS SANCHEZ (DNI ...,NaN,NaN,NaN,0,NaN,INSCRIPCION INSTITUTO NACIONAL DE LA PROPIEDAD...,inscripcion instituto nacional propiedad indus...,0.360556
365541,365411,Resolución,111,NaN,INSTITUTO NACIONAL DE LA PROPIEDAD INDUSTRIAL,2022-05-20,34928.0,2022-05-27,19.0,INSCRIPCION,INSTITUTO NACIONAL DE LA PROPIEDAD INDUSTRIAL,INSCRIBASE A LA SEÑORA MARIA ALEJANDRA RODRIGU...,NaN,NaN,NaN,0,NaN,INSCRIPCION INSTITUTO NACIONAL DE LA PROPIEDAD...,inscripcion instituto nacional propiedad indus...,0.358589
379297,380531,Resolución,86,NaN,INSTITUTO NACIONAL DE LA PROPIEDAD INDUSTRIAL,2023-03-03,35127.0,2023-03-10,19.0,INSCRIPCION,INSTITUTO NACIONAL DE LA PROPIEDAD INDUSTRIAL,INSCRIBASE AL SEÑOR JUAN EDUARDO VERA (DNI N° ...,NaN,NaN,NaN,0,NaN,INSCRIPCION INSTITUTO NACIONAL DE LA PROPIEDAD...,inscripcion instituto nacional propiedad indus...,0.357703
363183,363119,Resolución,30,NaN,INSTITUTO NACIONAL DE LA PROPIEDAD INDUSTRIAL,2022-03-31,34894.0,2022-04-05,14.0,INSCRIPCION,INSTITUTO NACIONAL DE LA PROPIEDAD INDUSTRIAL,INSCRIBASE EL SEÑOR DIEGO NICOLAS GONZALEZ (DN...,NaN,NaN,NaN,0,NaN,INSCRIPCION INSTITUTO NACIONAL DE LA PROPIEDAD...,inscripcion instituto nacional propiedad indus...,0.356858
365142,368066,Resolución,153,NaN,INSTITUTO NACIONAL DE LA PROPIEDAD INDUSTRIAL,2022-05-12,34962.0,2022-07-15,30.0,INSCRIPCION,INSTITUTO NACIONAL DE LA PROPIEDAD INDUSTRIAL,INSCRIBASE AL SEÑOR WALTER HORACIO MARTINEZ (D...,NaN,NaN,NaN,0,NaN,INSCRIPCION INSTITUTO NACIONAL DE LA PROPIEDAD...,inscripcion instituto nacional propiedad indus...,0.354887
363268,363189,Resolución,35,NaN,INSTITUTO NACIONAL DE LA PROPIEDAD INDUSTRIAL,2022-04-04,34895.0,2022-04-06,18.0,INSCRIPCION,INSTITUTO NACIONAL DE LA PROPIEDAD INDUSTRIAL,INSCRIBASE AL SEÑOR ALEJANDRO JAVIER VEGA (DNI...,NaN,NaN,NaN,0,NaN,INSCRIPCION INSTITUTO NACIONAL DE LA PROPIEDAD...,inscripcion instituto nacional propiedad indus...,0.354865
379119,380233,Resolución,70,NaN,INSTITUTO NACIONAL DE LA PROPIEDAD INDUSTRIAL,2023-02-28,35122.0,2023-03-03,7.0,INSCRIPCION,INSTITUTO NACIONAL DE LA PROPIEDAD INDUSTRIAL,INSCRIBASE AL SEÑOR RAFAEL CORNEJO (DNI N° 33....,NaN,NaN,NaN,0,NaN,INSCRIPCION INSTITUTO NACIONAL DE LA PROPIEDAD...,inscripcion instituto nacional propiedad indus...,0.354517
